In [1]:
#필요한 library 들을 load 합니다.
import os 

import random
import numpy as np
from tqdm import tqdm

import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision.transforms as transforms

from multiprocessing import cpu_count
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch.nn import CrossEntropyLoss
from torchvision.models import efficientnet_v2_s
from torchvision.models import efficientnet_b0
from torchvision.models.efficientnet import EfficientNet_V2_S_Weights
from torchvision.models import EfficientNet_B0_Weights
from sklearn.model_selection import train_test_split

from PIL import Image

In [2]:
USE_CUDA = torch.cuda.is_available() # GPU를 사용가능하면 True, 아니라면 False를 리턴
device = torch.device("cuda" if USE_CUDA else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용
print("다음 기기로 학습합니다:", device)

# from google.colab import drive
# drive.mount('/content/jdrive/')

다음 기기로 학습합니다: cpu


In [5]:
%cd /data/

[WinError 2] 지정된 파일을 찾을 수 없습니다: '/data/'
C:\Users\SSAFY\Desktop\bamboo_picture


In [32]:
!rm -rf /content/image

In [33]:
!rm -rf /content/__MACOSX

In [34]:
!unzip -qq /content/jdrive/MyDrive/Colab\ Notebooks/image.zip


In [3]:
transform = transforms.Compose([
    transforms.ToTensor(), #이미지 데이터를 tensor 데이터 포멧으로 바꾸어줍니다.
    transforms.Resize([224,224]), #이미지의 크기가 다를 수 있으니 크기를 통일해 줍니다.
    
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.Normalize(mean=(0.52273315, 0.52848613, 0.5472654), std=(0.21402003, 0.202031, 0.22438332)) # 이미지들의 픽셀 r, g, b의 평균 => 아래 코드에 구하는 부분있으니 참고할 것
])

In [4]:
train_data = datasets.ImageFolder(root='./data/image2/',transform=transform)
train_data

Dataset ImageFolder
    Number of datapoints: 3045
    Root location: ./data/image2/
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=[224, 224], interpolation=bilinear, max_size=None, antialias=None)
               RandomHorizontalFlip(p=0.5)
               RandomVerticalFlip(p=0.5)
               Normalize(mean=(0.52273315, 0.52848613, 0.5472654), std=(0.21402003, 0.202031, 0.22438332))
           )

In [5]:
train_idx, valid_idx = train_test_split(np.arange(len(train_data)), test_size=0.2, train_size = 0.8,
                                        random_state=42, shuffle=True, stratify=train_data.targets)

In [6]:
batch_size = 32 #batchsize는 2의배수롤 설정하시면 되는데 데이터 크기가 클수록 batch_size도 키워주시면 됩니다. 보통 32,64 사용하시면됩니다. 이 이상키우시면 cuda가 메모리양을 감당못하고 꺼질수있습니다.
num_workers = 1

In [7]:
train_loader = DataLoader(train_data, batch_size=batch_size, 
                          sampler=SubsetRandomSampler(train_idx), num_workers=num_workers)
valid_loader = DataLoader(train_data, batch_size=batch_size, 
                          sampler=SubsetRandomSampler(valid_idx), num_workers=num_workers)

In [8]:
train_total = len(train_idx)
valid_total = len(valid_idx)

train_batches = len(train_loader)
valid_batches = len(valid_loader)

In [9]:
print('total train imgs :',train_total,'/ total train batches :', train_batches)
print('total valid imgs :',valid_total, '/ total valid batches :', valid_batches)

total train imgs : 2436 / total train batches : 77
total valid imgs : 609 / total valid batches : 20


In [ ]:
# import numpy as np

# meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x,_ in train_loader]
# stdRGB = [np.std(x.numpy(), axis=(1,2)) for x,_ in train_loader]

# meanR = np.mean([m[0] for m in meanRGB])
# meanG = np.mean([m[1] for m in meanRGB])
# meanB = np.mean([m[2] for m in meanRGB])

# stdR = np.mean([s[0] for s in stdRGB])
# stdG = np.mean([s[1] for s in stdRGB])
# stdB = np.mean([s[2] for s in stdRGB])

# print(meanR, meanG, meanB)
# print(stdR, stdG, stdB)

0.52273315 0.52848613 0.5472654
0.21402003 0.202031 0.22438332


In [10]:
class PretrainModel(nn.Module):
    def __init__(self, pretrained):
        super().__init__()
        self.pretrained = pretrained
        self.FC = nn.Linear(1000,6)
    
    def forward(self,x):
        x = self.pretrained(x)
        x = self.FC(x)
        return x

In [11]:
weights = EfficientNet_V2_S_Weights.IMAGENET1K_V1
pretrained = efficientnet_v2_s(weights = weights)

In [12]:
net = PretrainModel(pretrained).to(device)

In [13]:
criterion = CrossEntropyLoss().to(device)
#criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.AdamW(params=net.parameters(), lr=0.001)
#optimizer = torch.optim.SGD(params=net.parameters(), lr = 0.001)
epochs = 20

In [14]:
#pretrain model
for epoch in range(epochs):
    net.train()
    
    train_loss = 0 
    train_correct = 0
    
    tqdm_dataset = tqdm(train_loader)
    for x,y in tqdm_dataset:
        x = x.to(device)
        y = y.to(device)
        outputs = net(x)
        loss = criterion(outputs,y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_correct += predicted.eq(y).sum().item()
        
        tqdm_dataset.set_postfix({
            'Epoch': epoch + 1,
            'Loss': '{:06f}'.format(loss.item()),
        })

    train_loss = train_loss / train_batches
    train_acc = train_correct / train_total
    
    net.eval()
    
    valid_loss = 0 
    valid_correct = 0
    tqdm_dataset = tqdm(valid_loader)
    with torch.no_grad():
        for x,y in tqdm_dataset:
            x = x.to(device)
            y = y.to(device)
            
            outputs = net(x)
            loss = criterion(outputs, y)
            valid_loss += loss.item()
            _, predicted = outputs.max(1)
            valid_correct += predicted.eq(y).sum().item()
            
            tqdm_dataset.set_postfix({
                'Epoch': epoch + 1,
                'Loss': '{:06f}'.format(loss.item()),
            })
            
    valid_loss = valid_loss / valid_batches
    valid_acc = valid_correct / valid_total
        
    print('epochs',epoch+1, 'train loss',train_loss,'train acc', train_acc, 'valid loss',valid_loss, 'valid acc',valid_acc)



100%|██████████| 20/20 [00:46<00:00,  2.33s/it, Epoch=1, Loss=0.026810]


epochs 1 train loss 1.4278439051145082 train acc 0.4548440065681445 valid loss 1.3963690700940787 valid acc 0.4466338259441708


100%|██████████| 20/20 [00:50<00:00,  2.53s/it, Epoch=2, Loss=0.722407]


epochs 2 train loss 1.135394063088801 train acc 0.5890804597701149 valid loss 1.9963604182004928 valid acc 0.5796387520525451


100%|██████████| 20/20 [00:47<00:00,  2.37s/it, Epoch=3, Loss=7.670651]


...

epochs 18 train loss 0.393716188994321 train acc 0.867816091954023 valid loss 1.185584881901741 valid acc 0.6798029556650246


100%|██████████| 20/20 [00:47<00:00,  2.39s/it, Epoch=19, Loss=0.024522]


epochs 19 train loss 0.26286000569725965 train acc 0.9100985221674877 valid loss 1.1210145091637969 valid acc 0.6978653530377669


100%|██████████| 20/20 [00:48<00:00,  2.41s/it, Epoch=20, Loss=0.038912]

epochs 20 train loss 0.22202860128569912 train acc 0.9248768472906403 valid loss 1.1463076945394277 valid acc 0.6929392446633826


In [16]:
torch.save(net, 'model.pt')

In [17]:
torch.save(net.state_dict(), 'model_state_dict.pt')